In [19]:
import pandas as pd


dedup = pd.read_excel(r"..\result\dedup_df_ver2.xlsx")
dedup

,ENGY_ESNCNO,SEQ_NO,MGM_BLD_PK,UNIT_CD,USE_QTY_11_UNIT_SUM,USE_QTY_12_UNIT_SUM,USE_QTY_13_UNIT_SUM,E사용량 (kWh)_전기 (KWH),E사용량 (kWh)_가스 (KWH),E사용량 (kWh)_지역냉난방 (KWH)
0,700173961,4233,51170-1220,1,73707.0,NaN,NaN,67459.997167,42373.166910,0.0
1,700105394,4234,51210-2917,1,114045.0,NaN,NaN,166346.999320,279391.121151,0.0
2,700384564,4236,51190-1301,1,180499.0,NaN,NaN,245569.980371,0.000000,0.0
3,1210027971,10114,47210-6263,8,NaN,1.067584e+06,NaN,170233.009451,267789.052920,0.0
4,1500102602,10114,47210-6263,1,170317.0,NaN,NaN,170233.009451,267789.052920,0.0
...,...,...,...,...,...,...,...,...,...,...
2489,337820629,10000004793984,29170-34708,1,2819800.0,NaN,NaN,234567.023810,0.000000,0.0
2490,4000487944,10000004793985,29170-34725,8,NaN,5.141939e+07,NaN,180565.969594,130084.253957,0.0
2491,344806311,10000004793987,29170-34729,1,202900.0,NaN,NaN,130091.996727,0.000000,0.0
2492,4000901930,10000004793987,29170-34729,8,NaN,1.621958e+07,NaN,130091.996727,0.000000,0.0


In [20]:
import numpy as np


conversion_factors = {
    1: 1.0,         # 1: KWh 그대로
    2: 10.6962086,  # 2: N㎥ → kWh
    3: 1163.0,      # 3: Gcal → kWh
    4: 1000.0,      # 4: MWh → kWh
    6: 1.163,       # 6: Mcal → kWh
    8: 0.2778236    # 8: MJ → kWh
    # 나머지 (5, 7, 9, 10, 14)는 변환계수가 없으므로 생략합니다.
}


In [21]:
dedup.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2494 entries, 0 to 2493
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   ENGY_ESNCNO             2494 non-null   int64  
 1   SEQ_NO                  2494 non-null   int64  
 2   MGM_BLD_PK              2494 non-null   object 
 3   UNIT_CD                 2494 non-null   int64  
 4   USE_QTY_11_UNIT_SUM     1370 non-null   float64
 5   USE_QTY_12_UNIT_SUM     981 non-null    float64
 6   USE_QTY_13_UNIT_SUM     143 non-null    float64
 7   E사용량 (kWh)_전기 (KWH)     2494 non-null   float64
 8   E사용량 (kWh)_가스 (KWH)     2494 non-null   float64
 9   E사용량 (kWh)_지역냉난방 (KWH)  2494 non-null   float64
dtypes: float64(6), int64(3), object(1)
memory usage: 195.0+ KB


In [22]:
def convert_usage(row):
    # 원본 행 복사
    new_row = row.copy()
    
    # UNIT_CD 확인 (정수형으로 처리)
    unit_cd = int(row['UNIT_CD']) if pd.notnull(row['UNIT_CD']) else None
    factor = conversion_factors.get(unit_cd, None)
    
    # 변환계수가 존재하는 경우에만 진행
    if factor is not None:
        # 각 변수별로 개별적으로 처리하여 새 변수에 저장
        if pd.notnull(row['USE_QTY_11_UNIT_SUM']):  # 전기 사용량
            new_row['USE_QTY_11_UNIT_SUM_kWh'] = row['USE_QTY_11_UNIT_SUM'] * factor
        else:
            new_row['USE_QTY_11_UNIT_SUM_kWh'] = np.nan
            
        if pd.notnull(row['USE_QTY_12_UNIT_SUM']):  # 가스 사용량
            new_row['USE_QTY_12_UNIT_SUM_kWh'] = row['USE_QTY_12_UNIT_SUM'] * factor
        else:
            new_row['USE_QTY_12_UNIT_SUM_kWh'] = np.nan
            
        if pd.notnull(row['USE_QTY_13_UNIT_SUM']):  # 지역냉난방 사용량
            new_row['USE_QTY_13_UNIT_SUM_kWh'] = row['USE_QTY_13_UNIT_SUM'] * factor
        else:
            new_row['USE_QTY_13_UNIT_SUM_kWh'] = np.nan
    else:
        # 변환계수가 없는 경우 NaN으로 설정
        new_row['USE_QTY_11_UNIT_SUM_kWh'] = np.nan
        new_row['USE_QTY_12_UNIT_SUM_kWh'] = np.nan
        new_row['USE_QTY_13_UNIT_SUM_kWh'] = np.nan
    
    return new_row

dedup1 = dedup.apply(convert_usage, axis=1)
dedup1

,ENGY_ESNCNO,SEQ_NO,MGM_BLD_PK,UNIT_CD,USE_QTY_11_UNIT_SUM,USE_QTY_12_UNIT_SUM,USE_QTY_13_UNIT_SUM,E사용량 (kWh)_전기 (KWH),E사용량 (kWh)_가스 (KWH),E사용량 (kWh)_지역냉난방 (KWH),USE_QTY_11_UNIT_SUM_kWh,USE_QTY_12_UNIT_SUM_kWh,USE_QTY_13_UNIT_SUM_kWh
0,700173961,4233,51170-1220,1,73707.0,NaN,NaN,67459.997167,42373.166910,0.0,73707.0,NaN,NaN
1,700105394,4234,51210-2917,1,114045.0,NaN,NaN,166346.999320,279391.121151,0.0,114045.0,NaN,NaN
2,700384564,4236,51190-1301,1,180499.0,NaN,NaN,245569.980371,0.000000,0.0,180499.0,NaN,NaN
3,1210027971,10114,47210-6263,8,NaN,1.067584e+06,NaN,170233.009451,267789.052920,0.0,NaN,2.966001e+05,NaN
4,1500102602,10114,47210-6263,1,170317.0,NaN,NaN,170233.009451,267789.052920,0.0,170317.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2489,337820629,10000004793984,29170-34708,1,2819800.0,NaN,NaN,234567.023810,0.000000,0.0,2819800.0,NaN,NaN
2490,4000487944,10000004793985,29170-34725,8,NaN,5.141939e+07,NaN,180565.969594,130084.253957,0.0,NaN,1.428552e+07,NaN
2491,344806311,10000004793987,29170-34729,1,202900.0,NaN,NaN,130091.996727,0.000000,0.0,202900.0,NaN,NaN
2492,4000901930,10000004793987,29170-34729,8,NaN,1.621958e+07,NaN,130091.996727,0.000000,0.0,NaN,4.506182e+06,NaN


In [26]:
dedup1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2494 entries, 0 to 2493
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   ENGY_ESNCNO              2494 non-null   int64  
 1   SEQ_NO                   2494 non-null   int64  
 2   MGM_BLD_PK               2494 non-null   object 
 3   UNIT_CD                  2494 non-null   int64  
 4   USE_QTY_11_UNIT_SUM      1370 non-null   float64
 5   USE_QTY_12_UNIT_SUM      981 non-null    float64
 6   USE_QTY_13_UNIT_SUM      143 non-null    float64
 7   E사용량 (kWh)_전기 (KWH)      2494 non-null   float64
 8   E사용량 (kWh)_가스 (KWH)      2494 non-null   float64
 9   E사용량 (kWh)_지역냉난방 (KWH)   2494 non-null   float64
 10  USE_QTY_11_UNIT_SUM_kWh  1370 non-null   float64
 11  USE_QTY_12_UNIT_SUM_kWh  981 non-null    float64
 12  USE_QTY_13_UNIT_SUM_kWh  143 non-null    float64
dtypes: float64(9), int64(3), object(1)
memory usage: 253.4+ KB


In [27]:
import numpy as np

# 전기 (에너지 타입 11)
mask_11 = np.isclose(dedup1['USE_QTY_11_UNIT_SUM'], dedup1['USE_QTY_11_UNIT_SUM_kWh'], equal_nan=True)
match_count_11 = mask_11.sum()
mismatch_count_11 = (~mask_11).sum()

# 가스 (에너지 타입 12)
mask_12 = np.isclose(dedup1['USE_QTY_12_UNIT_SUM'], dedup1['USE_QTY_12_UNIT_SUM_kWh'], equal_nan=True)
match_count_12 = mask_12.sum()
mismatch_count_12 = (~mask_12).sum()

# 지역냉난방 (에너지 타입 13)
mask_13 = np.isclose(dedup1['USE_QTY_13_UNIT_SUM'], dedup1['USE_QTY_13_UNIT_SUM_kWh'], equal_nan=True)
match_count_13 = mask_13.sum()
mismatch_count_13 = (~mask_13).sum()

print("USE_QTY_11_UNIT_SUM vs USE_QTY_11_UNIT_SUM_kWh:")
print("  일치하는 행:", match_count_11)
print("  불일치하는 행:", mismatch_count_11)
print()
print("USE_QTY_12_UNIT_SUM vs USE_QTY_12_UNIT_SUM_kWh:")
print("  일치하는 행:", match_count_12)
print("  불일치하는 행:", mismatch_count_12)
print()
print("USE_QTY_13_UNIT_SUM vs USE_QTY_13_UNIT_SUM_kWh:")
print("  일치하는 행:", match_count_13)
print("  불일치하는 행:", mismatch_count_13)


USE_QTY_11_UNIT_SUM vs USE_QTY_11_UNIT_SUM_kWh:
  일치하는 행: 2494
  불일치하는 행: 0

USE_QTY_12_UNIT_SUM vs USE_QTY_12_UNIT_SUM_kWh:
  일치하는 행: 1535
  불일치하는 행: 959

USE_QTY_13_UNIT_SUM vs USE_QTY_13_UNIT_SUM_kWh:
  일치하는 행: 2353
  불일치하는 행: 141


In [29]:
# 1. 전기 (ENGY_type 11) 비교 - NaN 값 제외
mask_elec_valid = dedup1['USE_QTY_11_UNIT_SUM'].notna()
valid_elec_df = dedup1[mask_elec_valid]
mask_elec = valid_elec_df['USE_QTY_11_UNIT_SUM'] == valid_elec_df['E사용량 (kWh)_전기 (KWH)']
elec_matching_count = mask_elec.sum()
elec_non_matching_count = len(valid_elec_df) - elec_matching_count
# 일치하지 않는 경우 차이 계산
elec_diff = valid_elec_df[~mask_elec]['USE_QTY_11_UNIT_SUM'] - valid_elec_df[~mask_elec]['E사용량 (kWh)_전기 (KWH)']
elec_diff_mean = elec_diff.mean()

# 2. 가스 (ENGY_type 12) 비교 - NaN 값 제외
mask_gas_valid = dedup1['USE_QTY_12_UNIT_SUM'].notna()
valid_gas_df = dedup1[mask_gas_valid]
mask_gas = valid_gas_df['USE_QTY_12_UNIT_SUM'] == valid_gas_df['E사용량 (kWh)_가스 (KWH)']
gas_matching_count = mask_gas.sum()
gas_non_matching_count = len(valid_gas_df) - gas_matching_count
# 일치하지 않는 경우 차이 계산
gas_diff = valid_gas_df[~mask_gas]['USE_QTY_12_UNIT_SUM'] - valid_gas_df[~mask_gas]['E사용량 (kWh)_가스 (KWH)']
gas_diff_mean = gas_diff.mean()

# 3. 지역냉난방 (ENGY_type 13) 비교 - NaN 값 제외
mask_local_valid = dedup1['USE_QTY_13_UNIT_SUM'].notna()
valid_local_df = dedup1[mask_local_valid]
mask_local = valid_local_df['USE_QTY_13_UNIT_SUM'] == valid_local_df['E사용량 (kWh)_지역냉난방 (KWH)']
local_matching_count = mask_local.sum()
local_non_matching_count = len(valid_local_df) - local_matching_count
# 일치하지 않는 경우 차이 계산
local_diff = valid_local_df[~mask_local]['USE_QTY_13_UNIT_SUM'] - valid_local_df[~mask_local]['E사용량 (kWh)_지역냉난방 (KWH)']
local_diff_mean = local_diff.mean()

# 결과 출력
print("전기 (ENGY_type 11) 일치 여부 (NaN 제외):")
print(" - 유효한 행의 개수:", len(valid_elec_df))
print(" - 일치하는 행의 개수:", elec_matching_count)
print(" - 일치하지 않는 행의 개수:", elec_non_matching_count)
print(" - 일치하지 않는 행의 차이 평균:", elec_diff_mean)
print()
print("가스 (ENGY_type 12) 일치 여부 (NaN 제외):")
print(" - 유효한 행의 개수:", len(valid_gas_df))
print(" - 일치하는 행의 개수:", gas_matching_count)
print(" - 일치하지 않는 행의 개수:", gas_non_matching_count)
print(" - 일치하지 않는 행의 차이 평균:", gas_diff_mean)
print()
print("지역냉난방 (ENGY_type 13) 일치 여부 (NaN 제외):")
print(" - 유효한 행의 개수:", len(valid_local_df))
print(" - 일치하는 행의 개수:", local_matching_count)
print(" - 일치하지 않는 행의 개수:", local_non_matching_count)
print(" - 일치하지 않는 행의 차이 평균:", local_diff_mean)

# 일치하는 행 출력
print("\n전기 (ENGY_type 11) 일치하는 행:")
print(valid_elec_df[mask_elec])

print("\n가스 (ENGY_type 12) 일치하는 행:")
print(valid_gas_df[mask_gas])

print("\n지역냉난방 (ENGY_type 13) 일치하는 행:")
print(valid_local_df[mask_local])


전기 (ENGY_type 11) 일치 여부 (NaN 제외):
 - 유효한 행의 개수: 1370
 - 일치하는 행의 개수: 2
 - 일치하지 않는 행의 개수: 1368
 - 일치하지 않는 행의 차이 평균: 24798036.20265978

가스 (ENGY_type 12) 일치 여부 (NaN 제외):
 - 유효한 행의 개수: 981
 - 일치하는 행의 개수: 16
 - 일치하지 않는 행의 개수: 965
 - 일치하지 않는 행의 차이 평균: 210558790.62711358

지역냉난방 (ENGY_type 13) 일치 여부 (NaN 제외):
 - 유효한 행의 개수: 143
 - 일치하는 행의 개수: 1
 - 일치하지 않는 행의 개수: 142
 - 일치하지 않는 행의 차이 평균: -332417.57902503095

전기 (ENGY_type 11) 일치하는 행:
      ENGY_ESNCNO   SEQ_NO    MGM_BLD_PK  UNIT_CD  USE_QTY_11_UNIT_SUM  \
1516    141475892  3393003    11680-7409        1             615859.0   
2028    227222902  5775121  41135-122837        1             269340.0   

      USE_QTY_12_UNIT_SUM  USE_QTY_13_UNIT_SUM  E사용량 (kWh)_전기 (KWH)  \
1516                  NaN                  NaN             615859.0   
2028                  NaN                  NaN             269340.0   

      E사용량 (kWh)_가스 (KWH)  E사용량 (kWh)_지역냉난방 (KWH)  USE_QTY_11_UNIT_SUM_kWh  \
1516           458826.925                     0.0        

In [28]:
# 1. 전기 (ENGY_type 11) 비교 - NaN 값 제외
mask_elec_valid = dedup1['USE_QTY_11_UNIT_SUM_kWh'].notna()
valid_elec_df = dedup1[mask_elec_valid]
mask_elec = valid_elec_df['USE_QTY_11_UNIT_SUM_kWh'] == valid_elec_df['E사용량 (kWh)_전기 (KWH)']
elec_matching_count = mask_elec.sum()
elec_non_matching_count = len(valid_elec_df) - elec_matching_count
# 일치하지 않는 경우 차이 계산
elec_diff = valid_elec_df[~mask_elec]['USE_QTY_11_UNIT_SUM_kWh'] - valid_elec_df[~mask_elec]['E사용량 (kWh)_전기 (KWH)']
elec_diff_mean = elec_diff.mean()

# 2. 가스 (ENGY_type 12) 비교 - NaN 값 제외
mask_gas_valid = dedup1['USE_QTY_12_UNIT_SUM_kWh'].notna()
valid_gas_df = dedup1[mask_gas_valid]
mask_gas = valid_gas_df['USE_QTY_12_UNIT_SUM_kWh'] == valid_gas_df['E사용량 (kWh)_가스 (KWH)']
gas_matching_count = mask_gas.sum()
gas_non_matching_count = len(valid_gas_df) - gas_matching_count
# 일치하지 않는 경우 차이 계산
gas_diff = valid_gas_df[~mask_gas]['USE_QTY_12_UNIT_SUM_kWh'] - valid_gas_df[~mask_gas]['E사용량 (kWh)_가스 (KWH)']
gas_diff_mean = gas_diff.mean()

# 3. 지역냉난방 (ENGY_type 13) 비교 - NaN 값 제외
mask_local_valid = dedup1['USE_QTY_13_UNIT_SUM_kWh'].notna()
valid_local_df = dedup1[mask_local_valid]
mask_local = valid_local_df['USE_QTY_13_UNIT_SUM_kWh'] == valid_local_df['E사용량 (kWh)_지역냉난방 (KWH)']
local_matching_count = mask_local.sum()
local_non_matching_count = len(valid_local_df) - local_matching_count
# 일치하지 않는 경우 차이 계산
local_diff = valid_local_df[~mask_local]['USE_QTY_13_UNIT_SUM_kWh'] - valid_local_df[~mask_local]['E사용량 (kWh)_지역냉난방 (KWH)']
local_diff_mean = local_diff.mean()

# 결과 출력
print("전기 (ENGY_type 11) 일치 여부 (NaN 제외):")
print(" - 유효한 행의 개수:", len(valid_elec_df))
print(" - 일치하는 행의 개수:", elec_matching_count)
print(" - 일치하지 않는 행의 개수:", elec_non_matching_count)
print(" - 일치하지 않는 행의 차이 평균:", elec_diff_mean)
print()
print("가스 (ENGY_type 12) 일치 여부 (NaN 제외):")
print(" - 유효한 행의 개수:", len(valid_gas_df))
print(" - 일치하는 행의 개수:", gas_matching_count)
print(" - 일치하지 않는 행의 개수:", gas_non_matching_count)
print(" - 일치하지 않는 행의 차이 평균:", gas_diff_mean)
print()
print("지역냉난방 (ENGY_type 13) 일치 여부 (NaN 제외):")
print(" - 유효한 행의 개수:", len(valid_local_df))
print(" - 일치하는 행의 개수:", local_matching_count)
print(" - 일치하지 않는 행의 개수:", local_non_matching_count)
print(" - 일치하지 않는 행의 차이 평균:", local_diff_mean)

# 일치하는 행 출력
print("\n전기 (ENGY_type 11) 일치하는 행:")
print(valid_elec_df[mask_elec])

print("\n가스 (ENGY_type 12) 일치하는 행:")
print(valid_gas_df[mask_gas])

print("\n지역냉난방 (ENGY_type 13) 일치하는 행:")
print(valid_local_df[mask_local])


전기 (ENGY_type 11) 일치 여부 (NaN 제외):
 - 유효한 행의 개수: 1370
 - 일치하는 행의 개수: 2
 - 일치하지 않는 행의 개수: 1368
 - 일치하지 않는 행의 차이 평균: 24798036.20265978

가스 (ENGY_type 12) 일치 여부 (NaN 제외):
 - 유효한 행의 개수: 981
 - 일치하는 행의 개수: 16
 - 일치하지 않는 행의 개수: 965
 - 일치하지 않는 행의 차이 평균: 57930948.45407308

지역냉난방 (ENGY_type 13) 일치 여부 (NaN 제외):
 - 유효한 행의 개수: 143
 - 일치하는 행의 개수: 1
 - 일치하지 않는 행의 개수: 142
 - 일치하지 않는 행의 차이 평균: 82495831.35041158

전기 (ENGY_type 11) 일치하는 행:
      ENGY_ESNCNO   SEQ_NO    MGM_BLD_PK  UNIT_CD  USE_QTY_11_UNIT_SUM  \
1516    141475892  3393003    11680-7409        1             615859.0   
2028    227222902  5775121  41135-122837        1             269340.0   

      USE_QTY_12_UNIT_SUM  USE_QTY_13_UNIT_SUM  E사용량 (kWh)_전기 (KWH)  \
1516                  NaN                  NaN             615859.0   
2028                  NaN                  NaN             269340.0   

      E사용량 (kWh)_가스 (KWH)  E사용량 (kWh)_지역냉난방 (KWH)  USE_QTY_11_UNIT_SUM_kWh  \
1516           458826.925                     0.0           